# Simulate models

cameo uses and extends the model data structures defined by [cobrapy](https://opencobra.github.io/cobrapy/), our favorite COnstraints-Based Reconstruction and Analysis tool for Python. cameo is thus 100% compatible with cobrapy. For efficiency reasons, however, cameo implements its own simulation methods that take advantage of a more advanced solver interface.

## Primer: Constraint-Based Modeling

Constraint-based modeling is a powerful modeling framework for analyzing metabolism on the genome scale ([McCloskey et al., 2013](http://www.ncbi.nlm.nih.gov/pubmed/23632383)). For a model that encompasses $n$ reactions that involve $m$ metabolites, $\mathbf{S}$ is a matrix of dimension $m \times n$ that encodes the stoichiometry of the metabolic reaction system; it is usually referred to as stoichiometric matrix. Assuming that the system is in a steady state—the concentration of metabolites are constant—the system of flux-balances can be formulated as

$$
\begin{align}
\mathbf{S} \mathbf{v} = 0\,,
\end{align}
$$

where $\mathbf{v}$ is the vector of flux rates. With the addition of a biologically meaningful objective, flux capacity constraints, information about the reversibility of reactions under physiological conditions, an optimization problem can be formulated that can easily be solved using [linear programming](https://en.wikipedia.org/wiki/Linear_programming).


, e.g., maximimization of biomass production,Given the maximization of growth rate as one potential biological objective $v_{biomass}$, i.e., the flux of an artificial reaction that consumes biomass components in empirically determined proportions, and assuming that the cell is evolutionary optimized to achieve that objective, and incorporating knowledge about reaction reversibility,  uptake and secretion rates, and maximum flux capacities in the form of lower and uppers bounds ($\mathbf{v}_{lb}$ and $\mathbf{v}_{ub}$) on the flux variables $\mathbf{v}$, one can formulate and solve an optimization problem to identify an optimal set of flux rates using flux balance analysis (FBA):

$$
\begin{align}
 Max ~ & ~ Z_{obj} = \mathbf{c}^{T} \mathbf{v}\\
 \text{s.t.}~ & ~ \mathbf{S} \mathbf{v} = 0 \\
 ~ & ~ \mathbf{v}_{lb} \leq \mathbf{v} \leq \mathbf{v}_{ub} \,.
\end{align}
$$

## Flux Balance Analysis

Load a model.

In [5]:
from cameo import load_model
model = load_model('iJO1366')

In cameo, flux balance analysis can be performed with the function `fba`.

In [7]:
from cameo import fba
%time fba_result = fba(model)

CPU times: user 410 ms, sys: 6.43 ms, total: 416 ms
Wall time: 421 ms


Basically, `fba` calls `model.solve()` and wraps the optimization solution in a `FluxDistributionResult` object. The maximum objective values (corresponding to a maximum growth rate) can obtained throug `result.objective_value`.

In [8]:
fba_result.data_frame

,flux
DM_4crsol_c,0.000219
DM_5drib_c,0.000221
DM_aacald_c,0.000000
DM_amob_c,0.000002
...,...
ZN2t3pp,0.000000
ZN2tpp,0.000335
ZNabcpp,0.000000
Zn2tex,0.000335


## Parsimonious Flux Balance Analysis

Parsimonious flux balance analysis ([Lewis et al., 2010](http://www.ncbi.nlm.nih.gov/pubmed/20664636)), a variant of FBA, performs FBA in in a first step to determine the maximum objective value $Z_{obj}$, fixes it in form of an additional model constraint ($\mathbf{c}^{T} \mathbf{v} \ge Z_{obj}$), and then minimizes in a second optimization the $L_1$ norm of $\mathbf{v}$. The assumption behind pFBA is that cells try to minimize flux magnitude as well in order to keep protein costs low.

$$
\begin{align}
 Max ~ & ~ \lvert \mathbf{v} \rvert\\
 \text{s.t.}~ & ~ \mathbf{S} \mathbf{v} = 0 \\
 & ~ \mathbf{c}^{T} \mathbf{v} \ge Z_{obj} \\
 ~ & ~ \mathbf{v}_{lb} \leq \mathbf{v} \leq \mathbf{v}_{ub} \,.
\end{align}
$$

In cameo, pFBA can be performed with the function `pfba`.

In [9]:
from cameo import pfba
%time pfba_result = pfba(model)

CPU times: user 429 ms, sys: 19 ms, total: 448 ms
Wall time: 460 ms


The `objective_function` value is $\lvert \mathbf{v} \rvert$ ...

In [10]:
pfba_result.objective_value

699.0222751839472

... whis is significantly smaller than flux vector of the original FBA solution.

In [11]:
abs(fba_result.data_frame.flux).sum()

702.81946544045991

Setp 2: Simulate knockouts phenotypes
-----------------------------------

Although PFBA and FBA can be used to simulate the effect of knockouts, other methods have been proven more valuable for that task: MOMA and ROOM. In *cameo* we implement a linear version of MOMA.

*******************************************
Simulating knockouts:

* Manipulate the bounds of the reaction (or use the shorthand method knock_out)

In [12]:
model.reactions.PGI

Id,PGI
Name,Glucose-6-phosphate isomerase
Stoichiometry,g6p_c <=> f6p_c
Lower bound,-1000.000000
Upper bound,1000.000000


In [13]:
model.reactions.PGI.knock_out()
model.reactions.PGI

Id,PGI
Name,Glucose-6-phosphate isomerase
Stoichiometry,g6p_c --> f6p_c
Lower bound,0.000000
Upper bound,0.000000


* Simulate using different methods:

In [14]:
%time fba_knockout_result = fba(model)
fba_knockout_result[model.reactions.BIOMASS_Ec_iJO1366_core_53p95M]

CPU times: user 266 ms, sys: 4.96 ms, total: 271 ms
Wall time: 272 ms


0.9761293262947403

In [15]:
%time pfba_knockout_result = pfba(model)
pfba_knockout_result[model.reactions.BIOMASS_Ec_iJO1366_core_53p95M]

CPU times: user 374 ms, sys: 3.5 ms, total: 378 ms
Wall time: 379 ms


0.9761293262947374

MOMA and ROOM relly on a reference (wild-type) flux distribution and we can use the one previously computed.

**Parsimonious FBA references seem to produce better results using this methods**

In [16]:
from cameo.flux_analysis.simulation import room, lmoma

In [17]:
%time lmoma_result = lmoma(model, reference=pfba_result.fluxes)
lmoma_result[model.reactions.BIOMASS_Ec_iJO1366_core_53p95M]

CPU times: user 16.3 s, sys: 246 ms, total: 16.6 s
Wall time: 16.8 s


0.8724093536243601

In [ ]:
%time room_result = room(model, reference=pfba_result.fluxes)
room_result[model.reactions.BIOMASS_Ec_iJO1366_core_53p95M]